<a href="https://colab.research.google.com/github/drypuddle1220/Sports_Betting_Analytics/blob/main/NBA_Predictions(Approach_2%2C_DataClean_v2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install nba_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.4/279.4 kB 4.7 MB/s eta 0:00:00


In [26]:
# prompt: read the csv and make it into the dataframe: /content/games_post_2022.csv

import pandas as pd

# Assuming the CSV file is in the /content/ directory
file_path = '/content/games_post_2022.csv'
final_clean = pd.read_csv(file_path)

final_clean['avg_pace'] = -1
final_clean['agr_off'] = -1

In [30]:

from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Celtics, which contains their team IN
abb_bank = []
for k in range(len(nba_teams)):
  abb_bank.append( [team for team in nba_teams][k]['id'])
abb_bank

import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
game_list = []


for team_id in abb_bank:
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    # Append the retrieved DataFrame to the list
    game_list.append(gamefinder.get_data_frames()[0])

# Concatenate all DataFrames into a single DataFrame
games = pd.concat(game_list, ignore_index=True)

# Convert 'GAME_DATE' to datetime objects
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
# Filter games to include only those after 2018
games_post_2022 = games[(games['GAME_DATE'].dt.year > 2017) & (games['GAME_DATE'].dt.year < 2020)]
games_post_2022['Total_Pts'] = games_post_2022['PTS'] + (games_post_2022['PTS'] + games_post_2022['PLUS_MINUS'])
games_post_2022
games_bucket = []
year = {}
for game in games_post_2022['GAME_ID']:
  games_bucket.append(game)
  year[game] = str(games_post_2022[games_post_2022['GAME_ID'] == game]['GAME_DATE'].dt.year.values[0])
print(year)

<ipython-input-30-1993966e1c1f>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_post_2022['Total_Pts'] = games_post_2022['PTS'] + (games_post_2022['PTS'] + games_post_2022['PLUS_MINUS'])


{'0021900491': '2019', '0021900477': '2019', '0021900469': '2019', '0021900444': '2019', '0021900431': '2019', '0021900416': '2019', '0021900403': '2019', '0021900390': '2019', '0021900373': '2019', '0021900361': '2019', '0021900354': '2019', '0021900336': '2019', '0021900309': '2019', '0021900295': '2019', '0021900282': '2019', '0021900272': '2019', '0021900261': '2019', '0021900243': '2019', '0021900228': '2019', '0021900215': '2019', '0021900204': '2019', '0021900188': '2019', '0021900182': '2019', '0021900165': '2019', '0021900150': '2019', '0021900138': '2019', '0021900119': '2019', '0021900105': '2019', '0021900099': '2019', '0021900066': '2019', '0021900052': '2019', '0021900043': '2019', '0021900028': '2019', '0021900014': '2019', '0011900070': '2019', '0011900063': '2019', '0011900053': '2019', '0011900026': '2019', '0011900015': '2019', '1521900072': '2019', '1521900060': '2019', '1521900042': '2019', '1521900023': '2019', '1521900013': '2019', '0021801220': '2019', '00218012

In [32]:
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.library.parameters import LeagueID
import pandas as pd

def process_game(game, games_bucket, year):
    try:
        # Attempt to retrieve box score advanced data
        arena = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=game)
        arena = arena.get_data_frames()[0]


        arena['OFF_AGR'] = arena['E_OFF_RATING'] * (arena['MIN']/48)
        offensive = arena['OFF_AGR'].sum()

        arena['PACE_AGR'] = arena['E_PACE'] * (arena['MIN']/48)
        pace = arena['PACE_AGR'].mean()


        #plz change return variable later
        return (offensive, pace)

    except Exception as e:
        print(f"Error processing game {game}: {e}")
        return None


# Create a list to store results
results = []

for i, game in enumerate(games_bucket):
    try:
        # Validate game_id (optional, depending on your specific requirements)
        offensive, pace = process_game(game, games_bucket, year)

        if offensive is not None:
            # Store or update results
            final_clean.loc[games_post_2022['GAME_ID'] == game, 'agr_off'] = offensive

        if pace is not None:
            # Store or update results
            final_clean.loc[games_post_2022['GAME_ID'] == game, 'avg_pace'] = pace

    except Exception as e:
        print(f"Unexpected error for game {game}: {e}")

    # Progress tracking
    if i % 10 == 0:
        print(f"Iteration {i+1}/{len(game_id)}")


Error processing game 0021900491: unsupported operand type(s) for /: 'str' and 'int'
Unexpected error for game 0021900491: cannot unpack non-iterable NoneType object
Iteration 1/5826
Error processing game 0021900477: unsupported operand type(s) for /: 'str' and 'int'
Unexpected error for game 0021900477: cannot unpack non-iterable NoneType object
Error processing game 0021900469: unsupported operand type(s) for /: 'str' and 'int'
Unexpected error for game 0021900469: cannot unpack non-iterable NoneType object
Error processing game 0021900444: unsupported operand type(s) for /: 'str' and 'int'
Unexpected error for game 0021900444: cannot unpack non-iterable NoneType object
Error processing game 0021900431: unsupported operand type(s) for /: 'str' and 'int'
Unexpected error for game 0021900431: cannot unpack non-iterable NoneType object
Error processing game 0021900416: unsupported operand type(s) for /: 'str' and 'int'
Unexpected error for game 0021900416: cannot unpack non-iterable Non

KeyboardInterrupt: 